# Note
This script process our training data to fit the required structure for using YOLOv5. First, we mount the machine to our drive.

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/2022_趨勢_機器學習比賽/')

# Create validation set
We divide our dataset into training set and validation set. The list of the number of the validation set is at
OBJ_Train_Datasets/STAS_YOLOv5/images/val_set.txt

In [ ]:
import random
num_list=[f'{i:08}' for i in range(1053)] # We have 1052 training images.
random.Random(0).shuffle(num_list)
val_list=num_list[:105] #1/10 of training set
tr_list=num_list[105:]
print(val_list)

# **Copy the images**
We copy the training images to our target directories.

In [ ]:
import shutil
orginal_location='./OBJ_Train_Datasets/Train_Images/'
tr_img_location='./OBJ_Train_Datasets/STAS_YOLOv5/images/train/'
val_img_location='./OBJ_Train_Datasets/STAS_YOLOv5/images/val/'

for num in tr_list:
  img=orginal_location+f'{num}.jpg'
  shutil.copyfile(img, tr_img_location+f'{num}.jpg')
for num in val_list:
  img=orginal_location+f'{num}.jpg'
  shutil.copyfile(img, val_img_location+f'{num}.jpg')

# Create the txt file
We create the required text file for YOLOv5.

In [4]:
import numpy as np
import xml.etree.ElementTree as ET

def create_txt(mode,label=False): # 'tr' or 'val'
  if mode == 'tr':
    output_location = './OBJ_Train_Datasets/STAS_YOLOv5/labels/train/'
    l = tr_list
  elif mode == 'val':
    output_location = './OBJ_Train_Datasets/STAS_YOLOv5/labels/val/'
    l = val_list
  else:
    print('the input argument is wrong, please check!')
    return None

  # Read the YC labels
  if label:
    YC_label_path = './OBJ_Train_Datasets/Train_Images/cluster_result.txt'
    YC_img_list = np.genfromtxt(YC_label_path,usecols=0,unpack=True,skip_header=False,dtype=str)
    YC_count_list = np.genfromtxt(YC_label_path,usecols=1,unpack=True,skip_header=False,dtype=int)
    YC_label_list = np.genfromtxt(YC_label_path,usecols=2,unpack=True,skip_header=False,dtype=int)

  x_norm, y_norm = 1716, 942

  for num in l:
    ann_file=f'OBJ_Train_Datasets/Train_Annotations/{num}.xml'
    output=f'{output_location}{num}.txt'
    label_list=[]
    
    # Read the position of the STAS from the annotations file
    tree = ET.parse(ann_file)
    root = tree.getroot()
    STAS_position_dict={} # Iinitialize the dictionary
    count = 0 # Initialize the counting (meaningfule under the YC lebels case)
    for obj in root.findall('object'):
      xmin = int(obj.find('bndbox').find('xmin').text)
      ymin = int(obj.find('bndbox').find('ymin').text)
      xmax = int(obj.find('bndbox').find('xmax').text)
      ymax = int(obj.find('bndbox').find('ymax').text)
      
      width = np.round((xmax-xmin)/x_norm,6)
      height = np.round((ymax-ymin)/y_norm,6)
      x_cen = np.round((xmax+xmin)/(2*x_norm),6)
      y_cen = np.round((ymax+ymin)/(2*y_norm),6)

      if label:
        YC_label_index = np.where(YC_count_list[YC_img_list==num]==count)[0][0]
        YC_label = YC_label_list[YC_img_list==num][YC_label_index] - 1
        count+=1
        label_list.append(f'{YC_label} {x_cen} {y_cen} {width} {height} \n')
      else:
        label_list.append(f'0 {x_cen} {y_cen} {width} {height} \n')
    
    f = open(output,'w+')
    f.writelines(label_list)
    f.close()

  return None
create_txt('tr',label=False)
create_txt('val',label=False)

# **Create feature vectors manually**
In this block we design the feature vectors manually for the following clustering method.

In [ ]:
# Read the dataset and return the image as a 3D array and a dictionary contains 
# all important information listed in the annotation files.
# Be aware that 
# (1) the color order of the img array is BGR if you use cv2.imread.
# (2) the dimensions of the image are ordered in (y,x,color).
import cv2
import xml.etree.ElementTree as ET

def Read(img_num,read_img=True):
  img_file=f'OBJ_Train_Datasets/Train_Images/{img_num}.jpg'
  ann_file=f'OBJ_Train_Datasets/Train_Annotations/{img_num}.xml'
  tree = ET.parse(ann_file)
  root = tree.getroot()
  inf_dict={}
  count=0

  for obj in root.findall('object'):
    xmin=int(obj.find('bndbox').find('xmin').text)
    ymin=int(obj.find('bndbox').find('ymin').text)
    xmax=int(obj.find('bndbox').find('xmax').text)
    ymax=int(obj.find('bndbox').find('ymax').text)
    inf_dict[count]=[(xmin,ymin),(xmax,ymax)]
    count+=1

  inf_dict['STAS_num']=count
  inf_dict['xsize']=root.find('size').find('width').text
  inf_dict['ysize']=root.find('size').find('height').text
  inf_dict['zsize']=root.find('size').find('depth').text

  if read_img:
    img=cv2.imread(img_file)
    return img, inf_dict
  else:
    return inf_dict

In [ ]:
import random
import matplotlib.pyplot as plt

output_path = 'YC_feature_vectors.txt'

# Size, shape, neighborhood
img_num_list=[f'{i:08}' for i in range(1053)] # We have 1053 training images
# random.shuffle(img_num_list)
feature_list=[]
img_size=1716*942
extend = 10   # Use to recognize the surroudings of STASs.
# Number of STAS in each image, size and shape distibution
for img_num in img_num_list:
  print(f'Dealing with image {img_num} now....')
  img, inf = Read(img_num,read_img=True)
  for obj_num in range(inf['STAS_num']):
    # neighborhood. We count the pixel ratio of the white in surroundings.
    x_ind1,x_ind2=inf[obj_num][0][0],inf[obj_num][1][0]
    y_ind1,y_ind2=inf[obj_num][0][1],inf[obj_num][1][1]
    xmin=max(x_ind1-extend,0)
    ymin=max(y_ind1-extend,0)
    xmax=min(x_ind2+extend,1716)
    ymax=min(y_ind2+extend,942)
    img_sur = img[ymin:ymax,xmin:xmax,:].astype(float)
    img_sur[extend:-extend,extend:-extend,:] = np.nan
    index = np.nansum(img_sur,axis=2) < 650
    img_masked = img[ymin:ymax,xmin:xmax,:].astype('float')
    img_masked[index,:] = np.nan
    total_sur_area=img_sur[:,:,0].size-np.sum(np.isnan(img_sur[:,:,0]))
    white_sur_area=img_masked[:,:,0].size-np.sum(np.isnan(img_masked[:,:,0]))
    area_ratio = white_sur_area/total_sur_area

    xlen,ylen=x_ind2-x_ind1,y_ind2-y_ind1
    area=xlen*ylen/img_size
    shape=xlen/ylen

    feature_list.append(f'{img_num}_{obj_num} {area_ratio} {area} {shape} \n')

f = open(output_path,'w')
f.writelines(feature_list)
f.close();